In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx

df = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='ISO-8859-1')

In [ ]:
df['from_to'] = df[['User-ID', 'ISBN', 'Book-Rating']].apply(tuple, axis=1)
df.head()

In [ ]:
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(df['User-ID'], bipartite=0)
B.add_nodes_from(df['ISBN'], bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_weighted_edges_from(df['from_to'])

users = {n for n, d in B.nodes(data=True) if d['bipartite']==0}
books = set(B) - users

In [ ]:
# project bipartite graph onto user nodes keeping number of co-occurences, weighted degree
user_wt = nx.bipartite.weighted_projected_graph(B, users)

# project bipartite graph onto book nodes keeping number of co-occurences, weighted degree
book_wt = nx.bipartite.weighted_projected_graph(B, books, ratio=False)

In [ ]:
print(nx.info(B))

In [ ]:
#Code from textbook Social Network Analysis for Startups by Maksim Tsvetovat and Alexander Kouznetsov
def trim_edges(g, weight=1):
    g2=nx.Graph()
    for f, to, edata in g.edges(data=True):
        if edata['weight'] > weight:
            g2.add_edge(f,to,weight=edata['weight'])
    return g2

def island_method(g, iterations = 5):
    weights = [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    step = int((mx-mn)/iterations)
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx,step)]

cc = list(B.subgraph(c) for c in nx.connected_components(B))[0]
islands = island_method(cc)
#islands

#for i in islands:
#    print(i[0], len(i[1]), len(nx.connected_component_subgraphs(i[1])))
    

# use 4 for the threshold
#WW4 = trim_edges(WW, 4)

#plt.figure(figsize=(15,10))
#pos = nx.spring_layout(WW4)
#nx.draw_networkx_nodes(WW4, pos, node_color='yellow', node_size=500)
#nx.draw_networkx_edges(WW4, pos,  edge_color='gray')
#nx.draw_networkx_labels(WW4, pos, font_size=10, font_color ='blue')

#plt.show()